In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip3 install surprise # 처음 한번만 인스톨

In [3]:
ratings = pd.read_csv("./data/ml-latest-small/ratings.csv")

In [4]:
ratings = ratings.drop('timestamp', axis=1)

## 서프라이즈 패키지용 데이터셋 세팅

In [5]:
from surprise import Reader, Dataset

In [6]:
# 서프라이즈 패키지에게 rating의 최솟값, 최댓값 알려줌
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(ratings, reader)

## train, test 데이터 셋 세팅

In [7]:
from surprise.model_selection import train_test_split

In [8]:
r_train, r_test = train_test_split(data, test_size=0.1, random_state=42)

<br>

<br>

# KNNBasic 모델링 user-user

### 영화 하나만 예측해보기

In [9]:
from surprise import KNNBasic

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [10]:
# knn basic 모델 셋팅
sim_options = {'name': 'cosine',
              'min_support': 1,
              'user_based': True}

model = KNNBasic(k=10, min_k=3, sim_options=sim_options)

In [11]:
model.fit(r_train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [12]:
# test 데이터 실제값 확인 (user_id, item_id, rating)
r_test[:10]

[(325, 3863, 3.0),
 (1, 6, 4.0),
 (104, 4676, 3.0),
 (441, 4402, 5.0),
 (474, 2211, 3.0),
 (298, 8814, 0.5),
 (63, 115617, 5.0),
 (448, 380, 3.0),
 (182, 2022, 5.0),
 (68, 46970, 3.0)]

In [13]:
# 데이터 한 개 뽑아오기 (user_id, item_id, rating)
tr_only = r_test[2]
tr_only

(104, 4676, 3.0)

In [14]:
# 데이터 한개에 대한 모델의 예측값
uid = tr_only[0]
iid = tr_only[1]

tr_only_pred = model.predict(uid, iid) # 예측결과 전체 다 가져오기
tr_only_pred

Prediction(uid=104, iid=4676, r_ui=None, est=1.6820061369915678, details={'actual_k': 5, 'was_impossible': False})

In [15]:
tr_only_pred.est # 예측 '값'만 가져오기

1.6820061369915678

예측값 결과 설명
- uid : user_id
- iid : item_id
- r_ui : rating(실제값)
- est : 모델이 예측한 값
- actual_k : 지정해준 최소 최대값 사이에서 실제로 이 예측을 하기 위해 사용된 neighbor의 숫자, was_impossible이 True이면 이 정보는 나타나지 않음
- was_impossible : 지정해준 조건 내에서 예측이 가능했는지 불가능했는지 기록된 지표 True일 경우 조건내에서 예측값을 가져오는 것이 불가능했다는 표시
    - reason : 왜 조건 내에서 예측이 불가능했는지 이유를 알려주는 항목

In [16]:
# was_impossible = True인 경우
tr_only = r_test[0]
tr_only

(325, 3863, 3.0)

In [17]:
# 데이터 한개에 대한 모델의 예측값
uid = tr_only[0]
iid = tr_only[1]

tr_only_pred = model.predict(uid, iid) # 예측결과 전체 다 가져오기
tr_only_pred

Prediction(uid=325, iid=3863, r_ui=None, est=3.1999817408586377, details={'actual_k': 10, 'was_impossible': False})

was_impossible = True일 경우 ratings의 glabal_mean으로 예측.

In [18]:
tr_only_pred.est # 예측 '값'만 가져오기

3.1999817408586377

In [19]:
r_train.global_mean

3.5026225317348376

### r_test 리스트에 있는 영화 모두 예측해보기

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [20]:
sim_options = {'name': 'cosine',
              'min_support': 1,
              'user_based': True}

model = KNNBasic(k=10, min_k=3, sim_options=sim_options)

In [21]:
model.fit(r_train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [22]:
predictions = model.test(r_test)
predictions[:10]

[Prediction(uid=325, iid=3863, r_ui=3.0, est=3.1999817408586377, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=1, iid=6, r_ui=4.0, est=4.350761044575351, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=104, iid=4676, r_ui=3.0, est=1.6820061369915678, details={'actual_k': 5, 'was_impossible': False}),
 Prediction(uid=441, iid=4402, r_ui=5.0, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid=474, iid=2211, r_ui=3.0, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid=298, iid=8814, r_ui=0.5, est=2.232020032421871, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=63, iid=115617, r_ui=5.0, est=4.1513356395820376, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=448, iid=380, r_ui=3.0, est=3.550925566266186, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=182, iid=2

RMSE 확인해보기

In [23]:
from surprise.accuracy import rmse

In [24]:
rmse(predictions)

RMSE: 0.9854


0.9854225599381397

## KNNBasic 모델링 user-user(파라미터 튜닝)

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [67]:
sim_options = {'name': 'cosine',
              'min_support': 1,
              'user_based': True}

model = KNNBasic(k=40, min_k=5, sim_options=sim_options, verbose=True)

In [68]:
model.fit(r_train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [69]:
# 예측값
uid = str(15)
iid = str(4018)
model.predict(uid, iid)

Prediction(uid='15', iid='4018', r_ui=None, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'})

In [70]:
predictions = model.test(r_test)

RMSE 확인해보기

In [71]:
from surprise.accuracy import rmse

In [72]:
rmse(predictions)

RMSE: 0.9762


0.9761751339369014

<br>

<br>

# KNNBasic 모델링 item-item

### 영화 하나만 예측해보기

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [31]:
# knn basic 모델 셋팅
sim_options = {'name': 'cosine',
              'min_support': 1,
              'user_based': False}

model = KNNBasic(k=10, min_k=3, sim_options=sim_options)

In [32]:
model.fit(r_train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [33]:
# test 데이터 실제값 확인 (user_id, item_id, rating)
r_test[:10]

[(325, 3863, 3.0),
 (1, 6, 4.0),
 (104, 4676, 3.0),
 (441, 4402, 5.0),
 (474, 2211, 3.0),
 (298, 8814, 0.5),
 (63, 115617, 5.0),
 (448, 380, 3.0),
 (182, 2022, 5.0),
 (68, 46970, 3.0)]

In [41]:
# 데이터 한 개 뽑아오기 (user_id, item_id, rating)
tr_only = r_test[0]
tr_only

(325, 3863, 3.0)

In [42]:
# 데이터 한개에 대한 모델의 예측값
uid = tr_only[0]
iid = tr_only[1]

tr_only_pred = model.predict(uid, iid) # 예측결과 전체 다 가져오기
tr_only_pred

Prediction(uid=325, iid=3863, r_ui=None, est=3.8, details={'actual_k': 10, 'was_impossible': False})

In [43]:
tr_only_pred.est # 예측 '값'만 가져오기

3.8

예측값 결과 설명
- uid : user_id
- iid : item_id
- r_ui : rating(실제값)
- est : 모델이 예측한 값
- actual_k : 지정해준 최소 최대값 사이에서 실제로 이 예측을 하기 위해 사용된 neighbor의 숫자, was_impossible이 True이면 이 정보는 나타나지 않음
- was_impossible : 지정해준 조건 내에서 예측이 가능했는지 불가능했는지 기록된 지표 True일 경우 조건내에서 예측값을 가져오는 것이 불가능했다는 표시
    - reason : 왜 조건 내에서 예측이 불가능했는지 이유를 알려주는 항목

In [52]:
# was_impossible = True인 경우
tr_only = r_test[3]
tr_only

(441, 4402, 5.0)

In [53]:
# 데이터 한개에 대한 모델의 예측값
uid = tr_only[0]
iid = tr_only[1]

tr_only_pred = model.predict(uid, iid) # 예측결과 전체 다 가져오기
tr_only_pred

Prediction(uid=441, iid=4402, r_ui=None, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'})

was_impossible = True일 경우 ratings의 glabal_mean으로 예측.

In [54]:
tr_only_pred.est # 예측 '값'만 가져오기

3.5026225317348376

In [55]:
r_train.global_mean

3.5026225317348376

### r_test 리스트에 있는 영화 모두 예측해보기

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [56]:
sim_options = {'name': 'cosine',
              'min_support': 1,
              'user_based': False}

model = KNNBasic(k=10, min_k=3, sim_options=sim_options)

In [57]:
model.fit(r_train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [58]:
predictions = model.test(r_test)
predictions[:10]

[Prediction(uid=325, iid=3863, r_ui=3.0, est=3.8, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=1, iid=6, r_ui=4.0, est=4.400382986188536, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=104, iid=4676, r_ui=3.0, est=3.9, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=441, iid=4402, r_ui=5.0, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid=474, iid=2211, r_ui=3.0, est=3.5026225317348376, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid=298, iid=8814, r_ui=0.5, est=2.15, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=63, iid=115617, r_ui=5.0, est=4.05, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=448, iid=380, r_ui=3.0, est=2.25, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=182, iid=2022, r_ui=5.0, est=3.55, details={'actual_k': 10, 'was_impossible': 

RMSE 확인해보기

In [59]:
from surprise.accuracy import rmse

In [60]:
rmse(predictions)

RMSE: 1.0273


1.0272846143656624

## KNNBasic 모델링 item-item(파라미터 튜닝)

모델에 사용되는 파라미터 설명
- k : 비슷한 유저들 중에서 최대 몇 명까지의 점수를 참고할지 최대값 (기본 40)
- min_k : 비슷한 유저들이 별로 없을 경우 최소로 매칭될 k의 갯수 (기본 1)
- sim_options : 유사도 옵션
    - name : 사용할 유사도 종류('SVD', 'cosine', 'pearson', 'pearson-baseline' / 기본 'SVD')
    - min_support : 유사한 아이템 갯수가 충분하지 않을 경우 최소로 사용할 수 있는 갯수 (기본 1)
    - user_based : user 기반 추천인지 item 기반 추천인지(True:user-user, False:item-item / 기본 True)
    - sgrinkage : pearson_baseline 유사도를 구할때 옵션으로 사용하는 파라미터.
- verbose : 내부 프로세싱 출력

In [91]:
sim_options = {'name': 'cosine',
              'min_support': 1,
              'user_based': False}

model = KNNBasic(k=60, min_k=10, sim_options=sim_options, verbose=True)

In [92]:
model.fit(r_train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


RMSE 확인해보기

In [93]:
from surprise.accuracy import rmse

In [94]:
rmse(predictions)

RMSE: 0.9762


0.9761751339369014

### 그리드서치로 최적 파라미터 확인(기획 확인용)

In [75]:
from surprise.model_selection import GridSearchCV

In [86]:
sim_options = {
    "name" : ["cosine"]
    , "min_support" : [1]
    , "user_based" : [False]
}
param_grid = {"k":[10,20,30,40,50,60] , "min_k":[1,5,10,15,20,25,30], "sim_options": sim_options}

In [87]:
gs = GridSearchCV(KNNBasic, param_grid, measures=["rmse"], n_jobs=3)

In [88]:
gs.fit(data)

In [89]:
gs.best_score["rmse"]

0.9684420464376252

In [90]:
gs.best_params["rmse"]

{'k': 60,
 'min_k': 10,
 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}